In [1]:
import pandas as pd

In [6]:
#----------------------------------------------------------------
import pandas as pd
import numpy as np

# Load the CSV files into DataFrames
data_file = "/home/py/data/PREDICT_SALES_v1/data/processed/lgbm_predictions.csv"
df_lightgbm = pd.read_csv(data_file)

data_file = "/home/py/data/PREDICT_SALES_v1/data/processed/prophet_predictions.csv"
df_prophet = pd.read_csv(data_file)

# Make sure the date columns are in the correct datetime format
df_prophet['ds'] = pd.to_datetime(df_prophet['ds'])
df_lightgbm['date'] = pd.to_datetime(df_lightgbm['date'])

# Rename the 'ds' column in df_prophet to 'date' to match df_lightgbm
df_prophet.rename(columns={'ds': 'date'}, inplace=True)

# Merge the DataFrames
df_merged = pd.merge(df_lightgbm, df_prophet, on=['item_id', 'date'], how='left')

# create lgbm column that has the best predictor
# Define the predictor columns
predictor_cols = [
    'predict_cluster_volume_fs_sales',
    'predict_cluster_hac_s_sales',
    'predict_cluster_kmeans_fs_sales',
    'predict_cluster_seasonal_fs_sales',
    'predict_all_sales'
]

# Dictionary to store best predictor per item_id based on MAE
best_predictors = {}

# Group by item_id and determine best predictor
for item_id, group in df_merged.groupby('item_id'):
    mae_scores = {
        col: np.mean(np.abs(group[col] - group['sales']))
        for col in predictor_cols
    }
    best_predictor = min(mae_scores, key=mae_scores.get)
    best_predictors[item_id] = best_predictor

# Create 'lgbm' column by selecting the best predictor per item_id
df_merged['lgbm'] = df_merged.apply(lambda row: row[best_predictors[row['item_id']]], axis=1)


# Save the merged DataFrame to a new CSV file
data_file = "/home/py/data/PREDICT_SALES_v1/data/processed/merged.csv"
df_merged.to_csv(data_file, index=False)

In [8]:
print(df_merged.columns.tolist())

['date', 'item_id', 'sales', 'epoch_week', 'woy', 'moy', 'qoy', 'soy', 'year', 'epoch_week_sin', 'epoch_week_cos', 'copy_sales', 'ly', 'partition', 'lag_1', 'lag_4', 'lag_13', 'lag_52', 'build_4', 'build_13', 'build_52', 'roll_mean_4', 'roll_median_4', 'roll_std_4', 'roll_mean_13', 'roll_median_13', 'roll_std_13', 'roll_mean_52', 'roll_median_52', 'roll_std_52', 'is_high_outlier', 'item_id_encoded', 'life_to_date', 'holiday_flag', 'cluster_kmeans_fs', 'cluster_gaussian_f', 'cluster_hac_s', 'cluster_seasonal_fs', 'cluster_trend_f', 'cluster_volume_fs', 'predict_avg_sales', 'roll_max_4', 'roll_min_4', 'roll_max_13', 'roll_min_13', 'roll_max_52', 'roll_min_52', 'predict_cluster_volume_fs_sales', 'predict_cluster_hac_s_sales', 'predict_cluster_kmeans_fs_sales', 'predict_cluster_seasonal_fs_sales', 'predict_all_sales', 'yhat', 'lgbm']


### MAE summary

In [21]:
def compute_mae(group):
    mae_ly = np.mean(np.abs(group['sales'] - group['ly']))
    mae_yhat = np.mean(np.abs(group['sales'] - group['yhat']))
    mae_lgbm = np.mean(np.abs(group['sales'] - group['lgbm']))
    
    # Find the model with the lowest MAE
    best_model = min(
        [('ly', mae_ly), ('yhat', mae_yhat), ('lgbm', mae_lgbm)],
        key=lambda x: x[1]
    )[0]
    
    return pd.Series({
        'mae_ly': mae_ly,
        'mae_yhat': mae_yhat,
        'mae_lgbm': mae_lgbm,
        'best_model': best_model
    })

filtered_df = df_merged[(df_merged['epoch_week'] >= 2843) & (df_merged['epoch_week'] <= 2883)]

# Step 2: Group by item_id and apply the function
item_mae_summary = filtered_df.groupby('item_id').apply(compute_mae).reset_index()

# Step 3: Count how many items had each model as the best
model_counts = item_mae_summary['best_model'].value_counts()

# Output
print("Number of items best predicted by each model:")
print(model_counts)

Number of items best predicted by each model:
best_model
lgbm    5051
ly       998
yhat     327
Name: count, dtype: int64


/tmp/ipykernel_1345566/88417915.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  item_mae_summary = filtered_df.groupby('item_id').apply(compute_mae).reset_index()


### sum comparison

In [22]:
# Step 1: Group by item_id and calculate the total absolute error per model
# epoch_test_start = 2843
# epoch_test_end = 2883

def compute_total_error(group):
    sum_sales = group['sales'].sum()
    
    total_error_ly = abs(sum_sales - group['ly'].sum())
    total_error_yhat = abs(sum_sales - group['yhat'].sum())
    total_error_lgbm = abs(sum_sales - group['lgbm'].sum())
    
    # Find the model with the lowest total error
    best_model_by_sum = min(
        [('ly', total_error_ly), ('yhat', total_error_yhat), ('lgbm', total_error_lgbm)],
        key=lambda x: x[1]
    )[0]
    
    return pd.Series({
        'sum_sales': sum_sales,
        'total_error_ly': total_error_ly,
        'total_error_yhat': total_error_yhat,
        'total_error_lgbm': total_error_lgbm,
        'best_model_by_sum': best_model_by_sum
    })

# Step 2: Apply the function to each item
item_sum_comparison = filtered_df.groupby('item_id').apply(compute_total_error).reset_index()

# Step 3: Count how many times each model had the lowest total error
best_by_sum_counts = item_sum_comparison['best_model_by_sum'].value_counts()

# Output
print("Number of items best predicted by each model (by total error):")
print(best_by_sum_counts)

Number of items best predicted by each model (by total error):
best_model_by_sum
lgbm    4068
ly      1217
yhat    1091
Name: count, dtype: int64


/tmp/ipykernel_1345566/4240500362.py:27: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  item_sum_comparison = filtered_df.groupby('item_id').apply(compute_total_error).reset_index()


In [20]:
print(f"Min epoch:{df_merged['epoch_week'].min()}  Max epoch: {df_merged['epoch_week'].max()}")

Min epoch:2790  Max epoch: 2883


In [17]:
print(df_merged.columns.tolist)

<bound method IndexOpsMixin.tolist of Index(['date', 'item_id', 'sales', 'epoch_week', 'woy', 'moy', 'qoy', 'soy',
       'year', 'epoch_week_sin', 'epoch_week_cos', 'copy_sales', 'ly',
       'partition', 'lag_1', 'lag_4', 'lag_13', 'lag_52', 'build_4',
       'build_13', 'build_52', 'roll_mean_4', 'roll_median_4', 'roll_std_4',
       'roll_mean_13', 'roll_median_13', 'roll_std_13', 'roll_mean_52',
       'roll_median_52', 'roll_std_52', 'is_high_outlier', 'item_id_encoded',
       'life_to_date', 'holiday_flag', 'cluster_kmeans_fs',
       'cluster_gaussian_f', 'cluster_hac_s', 'cluster_seasonal_fs',
       'cluster_trend_f', 'cluster_volume_fs', 'predict_avg_sales',
       'roll_max_4', 'roll_min_4', 'roll_max_13', 'roll_min_13', 'roll_max_52',
       'roll_min_52', 'predict_cluster_volume_fs_sales',
       'predict_cluster_hac_s_sales', 'predict_cluster_kmeans_fs_sales',
       'predict_cluster_seasonal_fs_sales', 'predict_all_sales', 'yhat',
       'lgbm'],
      dtype='object')>

### overall predictors units off

In [23]:
# Step 2: Sum actual and predicted sales
sum_sales = filtered_df['sales'].sum()
sum_ly = filtered_df['ly'].sum()
sum_yhat = filtered_df['yhat'].sum()
sum_lgbm = filtered_df['lgbm'].sum()

# Step 3: Compute total unit error for each predictor
error_ly = abs(sum_sales - sum_ly)
error_yhat = abs(sum_sales - sum_yhat)
error_lgbm = abs(sum_sales - sum_lgbm)

# Step 4: Print results
print(f"Total units sold: {sum_sales:,.0f}")
print(f"LY prediction error: {error_ly:,.0f} units")
print(f"YHAT prediction error: {error_yhat:,.0f} units")
print(f"LGBM prediction error: {error_lgbm:,.0f} units")

# Optional: Which model was closest overall
best_model = min(
    [('ly', error_ly), ('yhat', error_yhat), ('lgbm', error_lgbm)],
    key=lambda x: x[1]
)
print(f"Best model by total units: {best_model[0]} (off by {best_model[1]:,.0f} units)")

Total units sold: 120,872,614
LY prediction error: 33,611,028 units
YHAT prediction error: 33,725,201 units
LGBM prediction error: 24,883,572 units
Best model by total units: lgbm (off by 24,883,572 units)
